<a href="https://colab.research.google.com/github/FaridSharaf/30-Days-of-Code/blob/master/Notebooks/Data%20Preprocessing/Data%20Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os

In [ ]:
cb_columns = ['Organization Name','Organization Name URL','Number of Contacts','Company Type',
              'Operating Status','Last Funding Type','Industries','Description',
              'Total Funding Amount','Total Funding Amount Currency','Total Funding Amount Currency (in USD)',
              'Founded Date','Founded Date Precision','LinkedIn','Last Funding Date','Founders','Number of Founders',
              'Headquarters Location', 'Headquarters Regions', 'Diversity Spotlight (US Only)', 'Estimated Revenue Range',
              'Exit Date','Exit Date Precision','Closed Date','Closed Date Precision','Website',
              'Twitter','Facebook','Contact Email','Number of Articles','Hub Tags','Full Description',
              'Phone Number','Investment Stage','Number of Investments','Number of Lead Investments','Number of Exits',
              'Number of Exits (IPO)','Number of Diversity Investments','Number of Portfolio Organizations','Accelerator Program Type',
              'Accelerator Application Deadline','Accelerator Duration (in weeks)','Investor Type','Industry Groups','Number of Employees',
              'Funding Status','Number of Funding Rounds','Last Funding Amount','Last Funding Amount Currency',
              'Last Funding Amount Currency (in USD)','Last Equity Funding Amount','Last Equity Funding Amount Currency','Last Equity Funding Amount Currency (in USD)',
              'Last Equity Funding Type','Total Equity Funding Amount','Total Equity Funding Amount Currency','Total Equity Funding Amount Currency (in USD)',
              'Number of Investors','Number of Lead Investors','Top 5 Investors','Number of Acquisitions','Acquisition Status','Acquisition Terms',
              'Acquisition Type','Price','Price Currency','Price Currency (in USD)','Announced Date',
              'Announced Date Precision','Acquired by','Acquired by URL','Transaction Name','Transaction Name URL',
              'Last Leadership Hiring Date','Last Layoff Mention Date','CB Rank (Organization)','CB Rank (Company)','SEMrush - Monthly Visits',
              'SEMrush - Average Visits (6 months)','SEMrush - Monthly Visits Growth','SEMrush - Visit Duration','SEMrush - Visit Duration Growth',
              'SEMrush - Page Views / Visit','SEMrush - Page Views / Visit Growth','SEMrush - Bounce Rate','SEMrush - Bounce Rate Growth',
              'SEMrush - Global Traffic Rank','SEMrush - Monthly Rank Change (#)','SEMrush - Monthly Rank Growth','BuiltWith - Active Tech Count','Apptopia - Number of Apps',
              'Apptopia - Downloads Last 30 Days','G2 Stack - Total Products Active','IPqwery - Patents Granted','IPqwery - Trademarks Registered',
              'IPqwery - Most Popular Patent Class','IPqwery - Most Popular Trademark Class','Aberdeen - IT Spend','Aberdeen - IT Spend Currency','Aberdeen - IT Spend Currency (in USD)',
              'Actively Hiring', 'founded_year', 'Founders_ID', 'founders_id_list', 'crunchbase_scraping_status', 'linkedin_scraping_status']

linkedin_columns = ['total_management_exp', 'total_entrepreneurial_epx', 
                    'total_technical_exp', 'total_consulting_exp',
                    'total_investment_banking_exp', 'sum_Bachelor_degree', 
                    'sum_master_degree', 'sum_MBA_degree', 'sum_PhD_degree',
                    'avg_top_university', 'avg_technical_education', 'avg_management_education']

trend_index_columns = ['keywords', 'Industry trend', 'Product trend']

twitter_columns = ["tweets_count", "avg_replies", "avg_retweets", "avg_likes",  "avg_lengths", 
                   "num_negative_tweets", "avg_negative_emotions", "num_positive_tweets", 
                   "avg_positive_emotions", "total_num_retweets_by_venture", "langs"]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/startups_data.csv')

In [ ]:
df['Founders_ID'].fillna('nan', inplace = True)

In [ ]:
df['Operating Status'].value_counts()

Split founders string to a  list of founders

In [ ]:
def flatten(founders):
    flat_founders = []
    for founder in founders:
        if isinstance(founder, list):
            flat_founders.extend(flatten(founder))
        else:
            flat_founders.append(founder)
    return flat_founders

In [ ]:
def split_founders(founders):
    founders_list = []
    if ',' in founders:
        founders_list.append(founders.replace(" ", "").split(','))
        founders_list = flatten(founders_list)
    else:
        founders_list.append(founders)
    return founders_list

In [ ]:
df['founders_id_list'] = df['Founders_ID'].apply(split_founders)
df

Check fully scraped companies 

In [ ]:
crunchbase_scraped_files_path = '/content/drive/MyDrive/1_Farid/1_startup_survivability/data/crunchbase/links'
linkedin_scraped_files_path = '/content/drive/MyDrive/1_Farid/1_startup_survivability/data/Linkedin/linkedin_profiles'

crunchbase_scraped_files = os.listdir('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/crunchbase/links')
linkedin_scraped_files = os.listdir('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/Linkedin/linkedin_profiles')

# Extract Fully Scraped Companies

In [ ]:
def mapping(users, files_list):
    scraping_status = 'Failed'
    n = len(users)
    scraped_users = 0
    for user in users:
        if user + '.json' in files_list:
            scraped_users = scraped_users + 1
        else:
            scraping_status = 'Failed'

    if scraped_users == n:
        scraping_status = 'Done'
    return scraping_status

In [ ]:
df['crunchbase_scraping_status'] = df.apply(lambda x: mapping(x['founders_id_list'], crunchbase_scraped_files), axis = 1)
# df.head(3)

In [ ]:
df['linkedin_scraping_status'] = df.apply(lambda x: mapping(x['founders_id_list'], linkedin_scraped_files), axis = 1)
# df.head(3)

In [ ]:
df['linkedin_scraping_status'].value_counts()

In [ ]:
df['crunchbase_scraping_status'].value_counts()

In [ ]:
linkedin_scraped_df = df[df['linkedin_scraping_status'] == 'Done'].copy()

In [ ]:
crunchbase_scraped_df = df[df['crunchbase_scraping_status'] == 'Done'].copy()

In [ ]:
len(list(df[df['crunchbase_scraping_status'] == 'Failed']['founders_id_list']))

# Extract Non-scraped Crunchbase Users

In [ ]:
non_scraped_crunchbase_users = df[df['crunchbase_scraping_status'] == 'Failed']['founders_id_list']
non_scraped_crunchbase_users[:10]

In [ ]:
def flatten(founders):
    all_founders = []
    flat_founders = []
    for founder in founders:
        if ',' in founder:
            all_founders.append(founder.replace(" ", '').split(','))
        else:
            all_founders.append(founder)

    for founder in all_founders:
        if isinstance(founder, list):
            flat_founders.extend(flatten(founder))
        else:
            flat_founders.append(founder)
    return flat_founders

In [ ]:
non_scraped_crunchbase_users = flatten(non_scraped_crunchbase_users)
non_scraped_crunchbase_users[:10]

In [ ]:
len(non_scraped_crunchbase_users)

In [ ]:
import pickle

In [ ]:
lst = np.array_split(non_scraped_crunchbase_users, 2)

In [ ]:
len(lst)

In [ ]:
i = 1
for pic in lst:
    with open(f'/content/drive/MyDrive/1_Farid/temp/cb_nonscraped/users_list{i}', 'wb') as output_file:
        pickle.dump(pic, output_file)
    i = i + 1

In [ ]:
# with open(f'/content/drive/MyDrive/1_Farid/temp/cb_nonscraped/users_list1', 'rb') as input_file:
#         lsls = pickle.load(input_file)

In [ ]:
# lsls[:10]

# Merge Data

In [ ]:
all_cols = cb_columns + trend_index_columns + twitter_columns + linkedin_columns

In [ ]:
raw_df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/raw_startups.csv')
# raw_df = df.copy()

In [ ]:
linkedin_df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/linkedin_data.csv', usecols = ['Organization Name URL'] + linkedin_columns)

In [ ]:
trend_df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/trend_index_data.csv', usecols = ['Organization Name URL'] + trend_index_columns)

In [ ]:
twitter_df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/twitter_data.csv', usecols = ['Organization Name URL'] + twitter_columns)

In [ ]:
startups = pd.merge(raw_df, trend_df, on = 'Organization Name URL', how= 'left')

In [ ]:
startups = pd.merge(startups, twitter_df, on = 'Organization Name URL', how= 'left')

In [ ]:
startups = pd.merge(startups, linkedin_df, on = 'Organization Name URL', how= 'left')

In [ ]:
startups.to_csv('/content/', index =False, columns = all_cols)

# Fix Companies Label

In [ ]:
df = pd.read_csv('../../Data/startups_data.csv')

In [ ]:
fixed_companies = pd.read_csv('/content/drive/MyDrive/1_Farid/temp/companies/labeled_companies - original_open.csv')

In [ ]:
fixed_companies['SocialMedia Status'].value_counts()

In [ ]:
# fixed_companies.loc[(~pd.isna(fixed_companies['SocialMedia Status'])) & (fixed_companies['SocialMedia Status'] != "Active") & (fixed_companies['SocialMedia Status'] != "Closed")]

In [ ]:
open_fixed_companies = fixed_companies[fixed_companies['SocialMedia Status'] == 'Active']['Organization Name URL'].values

In [ ]:
closed_fixed_companies = fixed_companies[fixed_companies['SocialMedia Status'] == 'Closed']['Organization Name URL'].values

In [ ]:
# df

In [ ]:
def modify_companies(row, lis):
    if row['Organization Name URL'] in lis:
        row['Operating Status'] = "Closed"
    return row

In [ ]:
df = df.apply(lambda x: modify_companies(x, closed_fixed_companies), axis = 1).copy()

In [ ]:
df['Operating Status'].value_counts()

In [ ]:
linkedin_df = df[df['linkedin_scraping_status'] == "Failed"].copy()

In [ ]:
closed_list = linkedin_df[linkedin_df['Operating Status'] == 'Closed']['Founders_ID']

In [ ]:
linkedin_df['founders_id_list'] = linkedin_df['Founders_ID'].apply(split_founders)
linkedin_df

In [ ]:
len(closed_list)

In [ ]:
closed_list = flatten(closed_list)
len(closed_list)

In [ ]:
closed_list

In [ ]:
    with open(f'../lost_linkedin', 'wb') as output_file:
        pickle.dump(closed_list, output_file)

In [ ]:
df.to_csv('../../startups_data.csv', index = False)

# Acquired Companies

In [ ]:
def modify_acquired_companies(row):
    if (row['Operating Status'] == "Closed") and (row['Acquisition Status'] == "Was Acquired"):
        row['Operating Status'] = "Active"
    return row

In [ ]:
df = df.apply(modify_acquired_companies, axis = 1).copy()

In [ ]:
df['Operating Status'].value_counts()

In [ ]:
df.to_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/startups_data.csv', index = False)

# Sample Extraction

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/startups_data.csv')

In [ ]:
df['Operating Status'].value_counts()

In [ ]:
closed_companies = df[df['Operating Status'] == "Closed"].copy()

In [ ]:
open_companies = df.loc[(df['Operating Status'] == 'Active') & (df['linkedin_scraping_status'] == 'Done')].sample(2000)

In [ ]:
startups_sample = pd.concat([open_companies, closed_companies], ignore_index = True, sort = False)

In [ ]:
startups_sample.to_csv('/content/drive/MyDrive/1_Farid/1_startup_survivability/data/startups/startups_sample.csv', index = False)